# 대구 주차장 변수
- 지역별 평균/합으로 생성

In [ ]:
import pandas as pd

In [ ]:
df_parking = pd.read_csv('./대구 주차장 정보.csv',encoding='euc-kr')

## 데이터 전처리

In [ ]:
df_parking['데이터기준일자'].sort_values()

683    2019-06-01
695    2019-06-01
694    2019-06-01
693    2019-06-01
692    2019-06-01
          ...    
130    2020-03-17
129    2020-03-17
128    2020-03-17
138    2020-03-17
164    2020-03-17
Name: 데이터기준일자, Length: 871, dtype: object

In [ ]:
df_parking['소재지지번주소']

0      대구광역시 남구 봉덕동 1002-69
1        대구광역시 남구 봉덕동 687-8
2      대구광역시 남구 대명동 1803-10
3         대구광역시 남구 봉덕동 1256
4        대구광역시 남구 봉덕동 976-5
               ...         
866    대구광역시 중구 동인동1가 330-8
867       대구광역시 중구 수창동 95-1
868       대구광역시 중구 봉산동 38-2
869      대구광역시 중구 삼덕동2가 246
870        대구광역시 중구 동산동 194
Name: 소재지지번주소, Length: 871, dtype: object

In [ ]:
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
df_parking[['시','군','구']] = df_parking['소재지지번주소'].str.extract(location_pattern).iloc[:,:3]
df_parking = df_parking.drop(columns='소재지지번주소')

## Encoding 및 파생 변수 처리

In [ ]:
df_1 = pd.get_dummies(df_parking['주차장구분']).astype(int)
df_1

,공영,민영
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
866,0,1
867,0,1
868,0,1
869,0,1


In [ ]:
df_2 = pd.get_dummies(df_parking['주차장유형']).astype(int)
df_2

,노상,노외
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
866,0,1
867,0,1
868,0,1
869,0,1


In [ ]:
df_3 = pd.get_dummies(df_parking['급지구분']).astype(int)
df_3

,1,2,3
0,1,0,0
1,1,0,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
866,1,0,0
867,1,0,0
868,1,0,0
869,1,0,0


In [ ]:
df_parking = pd.concat([df_parking, df_1, df_2, df_3], axis=1)
df_parking = df_parking.drop(columns=['주차장구분','주차장유형','급지구분'])

In [ ]:
df_parking.columns

Index([    '주차장관리번호',        '주차장명',    '소재지도로명주소',       '주차구획수',
            '부제시행구분',        '운영요일',    '평일운영시작시각',    '평일운영종료시각',
         '토요일운영시작시각',   '토요일운영종료시각',   '공휴일운영시작시각',   '공휴일운영종료시각',
              '요금정보',      '주차기본시간',      '주차기본요금',      '추가단위시간',
            '추가단위요금', '1일주차권요금적용시간',     '1일주차권요금',      '월정기권요금',
              '결제방법',        '특기사항',          '위도',          '경도',
           '데이터기준일자',           '시',           '군',           '구',
                '공영',          '민영',          '노상',          '노외',
                   1,             2,             3],
      dtype='object')

In [ ]:
sr_1 = (pd.to_datetime(df_parking['평일운영종료시각']) - pd.to_datetime(df_parking['평일운영시작시각'])).dt.seconds
sr_2 = (pd.to_datetime(df_parking['토요일운영종료시각']) - pd.to_datetime(df_parking['토요일운영시작시각'])).dt.seconds
sr_3 = (pd.to_datetime(df_parking['공휴일운영종료시각']) - pd.to_datetime(df_parking['공휴일운영시작시각'])).dt.seconds

In [ ]:
df_4 = pd.DataFrame(data=zip(sr_1,sr_2,sr_3),
             columns=['평일운영시간','토요일운영시간','공휴일운영시간'])

In [ ]:
df_parking = pd.concat([df_parking, df_4], axis=1)
df_parking = df_parking.drop(columns=[
    '평일운영종료시각','평일운영시작시각',
    '토요일운영종료시각','토요일운영시작시각',
    '공휴일운영종료시각','공휴일운영시작시각',
    '운영요일'])

In [ ]:
df_5 = pd.get_dummies(df_parking['요금정보']).astype(int)
df_5

,무료,유료,혼합
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
866,0,1,0
867,0,1,0
868,0,1,0
869,0,1,0


In [ ]:
df_parking = pd.concat([df_parking, df_5], axis=1)
df_parking = df_parking.drop(columns=['요금정보'])

In [ ]:
df_parking.columns

Index([    '주차장관리번호',        '주차장명',    '소재지도로명주소',       '주차구획수',
            '부제시행구분',      '주차기본시간',      '주차기본요금',      '추가단위시간',
            '추가단위요금', '1일주차권요금적용시간',     '1일주차권요금',      '월정기권요금',
              '특기사항',          '위도',          '경도',     '데이터기준일자',
                 '시',           '군',           '구',          '공영',
                '민영',          '노상',          '노외',             1,
                   2,             3,      '평일운영시간',     '토요일운영시간',
           '공휴일운영시간',          '무료',          '유료',          '혼합'],
      dtype='object')

In [ ]:
df_parking = df_parking.drop(columns=[
    '주차장관리번호','주차장명','소재지도로명주소',
    '부제시행구분','특기사항','데이터기준일자'])

In [ ]:
df_parking['시군구'] = df_parking['시'] + ' ' + df_parking['군'] +  ' ' + df_parking['구']
df_parking = df_parking.drop(columns=['시','군','구'])

In [ ]:
parking_lots_count = df_parking['시군구'].value_counts()

In [ ]:
df_parking_by_place = df_parking.groupby('시군구').mean()

In [ ]:
df_parking_by_place['주차장수'] = parking_lots_count

In [ ]:
df_parking_by_place['시군구'] = df_parking_by_place.index

In [ ]:
df_parking_by_place.to_csv('./parking_by_place.csv',index=False,encoding='utf-8')